In [2]:
import seaborn as sns
from statistics import mean
import pandas as pd
from math import floor, sqrt
import numpy as np
from pathlib import Path
import more_itertools
from itertools import combinations
from collections import defaultdict
import re

In [3]:
# Step 1: Load context sanitized fields grouped at cutoffs
data_path = Path('../data/processed/')
file_name = Path("task_2_and_3.csv")

santized_fields_at_cutoffs = pd.read_csv(data_path/file_name)

data_path = Path('../data/interim/')
file_name = Path("qualitative_non_similar.csv")
qualitative_non_similar = pd.read_csv(data_path/file_name)

data_path = Path('../data/interim/')
file_name = Path("qualitative_similar_questions.csv")
qualitative_similar_questions = pd.read_csv(data_path/file_name)

# ... because the data triangulation process had manual components
# there can be accidental extra whitespace after some themes and
# capitalizations differences from auto-correct for same themes
non_similar_theme_assignment = 'Theme convereted areas'
similar_theme_assignment = 'Theme Converted Areas'

qualitative_non_similar[non_similar_theme_assignment] =\
    qualitative_non_similar[non_similar_theme_assignment].str.lower().str.strip()

qualitative_similar_questions[similar_theme_assignment] =\
    qualitative_similar_questions[similar_theme_assignment].str.lower().str.strip()

# We construct hashes over the questions because the data triangulation process
# did not include the primary keys and the back and forth left some
# empty rows.
#
# So we keep only those hashes present in santized_fields_at_cutoffs so that
# other data integrations don't see empty rows with hashes, etc.
santized_fields_at_cutoffs['hash'] =\
    pd.util.hash_array(
        santized_fields_at_cutoffs['value'].to_numpy()
)

qualitative_non_similar['hash'] =\
    pd.util.hash_array(
        qualitative_non_similar['value'].to_numpy()
)

qualitative_similar_questions['hash'] =\
    pd.util.hash_array(
        qualitative_similar_questions['value'].to_numpy()
)

use_only_these_hashes = set(santized_fields_at_cutoffs['hash'])
qualitative_similar_questions =\
    qualitative_similar_questions.query('hash in @use_only_these_hashes')

qualitative_non_similar =\
    qualitative_non_similar.query('hash in @use_only_these_hashes')

In [4]:
# Some quick unit tests ... 
hashes_in_qualitative = set(
    qualitative_similar_questions['hash']
).union(
    set(qualitative_non_similar['hash'])
)

hashes_in_santized_fields = set(santized_fields_at_cutoffs['hash'])

# Check that the number of hashes in santized data is equal (or greater) than
# those in the qualitative assignment.
#
# Note: that there some, very few, repeated questions, so rows != # hashes neccessarily
# Some forms have same questions, some forms repeate their questions, etc.
assert len(hashes_in_santized_fields) >= len(hashes_in_qualitative), "More qualitative assignments than available in santized fields!"

# Check the santized field hashes are at least a superset of the qualitative theme assignment ...
assert hashes_in_santized_fields.issuperset(hashes_in_qualitative), "Qualitative assignments contains different values than found in santized values!"

In [5]:
# Now we can assign themes to the fields with cutoffs and groups
# to do this we use a dictionary and applymap
hash_column_name = 'hash'
task_5i_ii = santized_fields_at_cutoffs.copy(deep=True)

# The qualitative data has ~21 minor Theme name typos that we fix up
# such can be the nature of mixed-method work
theme_typo_1 = "requested grant funding related related" # <-- repeated last word
theme_typo_2 = "organizational budgeting, revenue practices, and forecasts\n\n" # stray newlines
requested_grant_theme = 'Requested Grant Funding Related'
org_revenue_theme = 'Organizational Budgeting, Revenue Practices, and Forecasts'

set_these_indices =\
    qualitative_non_similar[non_similar_theme_assignment].str.lower() == theme_typo_1
qualitative_non_similar.loc[set_these_indices, non_similar_theme_assignment] = requested_grant_theme

set_these_indices =\
    qualitative_non_similar[non_similar_theme_assignment].str.lower() == theme_typo_2
qualitative_non_similar.loc[set_these_indices, non_similar_theme_assignment] = org_revenue_theme

In [6]:
the_themes =\
    dict(
        zip(
            qualitative_similar_questions.hash,
            qualitative_similar_questions[similar_theme_assignment]
        )
)

the_themes.update(
    dict(
        zip(
            qualitative_non_similar.hash,
            qualitative_non_similar[non_similar_theme_assignment]
        )
    )
)

task_5i_ii["Theme"] =\
    task_5i_ii[hash_column_name].apply(
    lambda the_hash: the_themes.get(the_hash, None)
)
task_5i_ii["Theme"] = task_5i_ii["Theme"].str.lower()

In [7]:
# There's a single question whose theme seems to be getting set to
# None and I'm not sure how.
#
# Easier to just assign its theme here ...
fix_this_hash = 10255192530720195144

fix_this_index = task_5i_ii.query('hash == @fix_this_hash').index
task_5i_ii.loc[fix_this_index, "Theme"] = 'Organizational Biographical and General Information'

In [8]:
task_5i_ii[['value','Theme']].sample(10, random_state=3)

,value,Theme
6466,FISCAL YEAR START DATE,"organizational budgeting, revenue practices, a..."
23903,Email Address,organizational biographical and general inform...
9310,ORGANIZATION MISSION STATEMENT,what the organization does
33081,"For Global Access purposes, please confirm tha...",what the organization does
24052,DEMOGRAPHICS: INCOME,project demographics/orientation/status
4004,Have you explored collaborating or partnering ...,collaborative partnerships and community support
8786,BRIEF SUMMARY OF REQUEST,what the organization does
22594,Does the applying organization have a 501(c)3 ...,requested grant funding related
22384,Briefly describe the proposed grant activities,what the organization does
29484,What are the expected outcomes of this project,what the organization does


In [9]:
# ... final unit test, check that Themes have been assigned to all rows
assert 0 == task_5i_ii[task_5i_ii['Theme'].isnull()].shape[0], "There are unassigned Themes!"

In [10]:
# for reporting purposes, print out value counts and percentages
print(
    task_5i_ii.query('the_cutoff == 0.11').Theme.str.lower().value_counts(normalize=True),
    "\n[Counts]\n",
    task_5i_ii.query('the_cutoff == 0.11').Theme.str.lower().value_counts(normalize=False),
    len(task_5i_ii.query('the_cutoff == 0.11'))
)

what the organization does                                        0.224210
requested grant funding related                                   0.201006
organizational budgeting, revenue practices, and forecasts        0.195415
organizational biographical and general information               0.179200
corporate delegation and oversight, organizational structure      0.051999
collaborative partnerships and community support                  0.047526
data handling, overview, measurement, evaluation and reporting    0.036064
miscellaneous                                                     0.033268
project demographics/orientation/status                           0.023483
how did you hear of us                                            0.003634
how has covid-19 impacted your work                               0.002237
alternative supports                                              0.001398
time spent filling out the form                                   0.000559
Name: Theme, dtype: float

In [11]:
task_5i_ii.to_csv('../data/processed/task_5i_ii.csv', index=False)